In [1]:
from pathlib import Path
INPUT_FILE = Path.cwd() / "day-07.txt"


In [2]:
with INPUT_FILE.open("r") as file:
    input = [line.strip() for line in file.readlines()]

input[0:16]

['$ cd /',
 '$ ls',
 '53302 chvtw.czb',
 '240038 dwhl.nrn',
 '124868 dwhl.vvb',
 'dir fml',
 'dir jbgpgvj',
 'dir qjphltd',
 'dir wlfprc',
 'dir zqvh',
 'dir zzmgz',
 '$ cd fml',
 '$ ls',
 'dir bztjtqg',
 '176916 crgzcmrt.jlr',
 '199024 gpnmqlr.rdb']

```text
Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?
```

In [3]:
class File:
    def __init__(self, name:str, size:int):
        self._name = name
        self._size = size
        return

    @property
    def name(self) -> str:
        return self._name

    @property
    def size(self) -> int:
        return self._size


In [19]:
from functools import reduce
from typing import Union

class Directory:
    def __init__(self, name:str, parent:"Directory"):
        self._name = name
        self._parent = parent
        self._contents = []
        return

    def add_entry(self, entry:Union[File, "Directory"]) -> None:
        self._contents.append(entry)
        return

    @property
    def name(self) -> str:
        return self._name

    @property
    def parent(self) -> "Directory":
        return self._parent

    @property
    def size(self) -> int:
        return reduce(lambda x, y: x + y.size, self._contents, 0)

    @property
    def path(self) -> str:
        return f"{self.parent.path}/{self.name}" if self.parent is not None else self.name


In [20]:
f = File("foo", 4096)
print(f.size)
d = Directory("src", Directory("/", None))
d.add_entry(f)
d.add_entry(File("bar", 8192))
print(d.size)
e = Directory("baz", d)
e.add_entry(File("boo", 1024))
e.add_entry(d)
print(e.size)

4096
12288
13312


In [21]:
root:Directory = Directory("/", None)
curdir:Directory = None
for output in input:
    if output.startswith("$"):
        tokens = output.split(" ", 2)
        command = tokens[1]
        target = tokens[2] if len(tokens) > 2 else None
        if command == "cd":
            if target == "..":
                curdir = curdir.parent
            elif target == "/":
                curdir = root
            else:
                curdir = list(filter(lambda p: p.name == target, curdir._contents))[0]
        elif command == "ls":
            pass    # ignore this, we will process the output by default below
        else:
            raise RuntimeError(f"Unknown command, '{command}'")
    else:
        size, name = output.split(" ")
        if size == "dir":
            curdir.add_entry(Directory(name, curdir))
        else:   # file entry
            curdir.add_entry(File(name, int(size)))


In [22]:
from typing import Dict

def get_sizes(curdir:Directory) -> Dict:

    catalog = {curdir.path: curdir.size}

    for obj in curdir._contents:
        if isinstance(obj, Directory):
            catalog.update(get_sizes(obj))

    return catalog


In [23]:
catalog = get_sizes(root)

total = 0
for path, size in catalog.items():
    if size <= 100_000:
        total += size

print(f"Part 1: total size of directories <= 100,000 bytes is {total}")

Part 1: total size of directories <= 100,000 bytes is 1447046


In [24]:
disk_size = 70_000_000
used_space = root.size
unused_space = disk_size - used_space
needed_space = 30_000_000 - unused_space

print(f"disk size    = {disk_size}")
print(f"used space   = {used_space}")
print(f"unused space = {unused_space}")
print(f"needed space = {needed_space}")

disk size    = 70000000
used space   = 40572957
unused space = 29427043
needed space = 572957


In [26]:
cname = root.path
csize = root.size
for path, size in catalog.items():
    if (size >= needed_space) and (size < csize):
        cname = path
        csize = size

print(f"Part 2: best candidate is '{cname}' at {csize} bytes.")


Part 2: best candidate is '//qjphltd/mgnl/vfrbjgdm/chvtw/bfwl/wlfprc/hsbg' at 578710 bytes.
